# Unsloth Fine-tuning on Google Colab

Train and fine-tune LLMs with Unsloth on Google Colab's free GPU.

**Before you start:**
1. Runtime → Change runtime type → GPU → T4 GPU (free tier)
2. Make a copy of this notebook to your Google Drive

**Total time:** ~10-15 minutes (setup + training)

## Step 1: Setup Environment

Install dependencies (takes ~5 minutes)

In [ ]:
%%capture
# Install dependencies in the correct order
!pip install --upgrade pip

# Core ML frameworks
!pip install "trl>=0.12.0" "peft>=0.13.0" "bitsandbytes>=0.45.0" "transformers[sentencepiece]>=4.46.0"

# PyTorch
!pip install torch==2.8.0 torchvision --index-url https://download.pytorch.org/whl/cu121

# Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# xformers
!pip install --no-deps "xformers>=0.0.32,<0.0.33" --index-url https://download.pytorch.org/whl/cu121

# Additional dependencies
!pip install datasets huggingface_hub accelerate sentencepiece protobuf python-dotenv

print("✅ Installation complete!")

## Step 2: Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/farhan-syah/unsloth-finetuning.git
%cd unsloth-finetuning

print("✅ Repository cloned!")

## Step 3: Configure Training

Edit these settings for your training run:

In [ ]:
# ============================================
# CONFIGURATION - Edit these settings
# ============================================

# Model Selection (choose based on use case)
LORA_BASE_MODEL = "unsloth/Qwen3-VL-2B-Instruct-unsloth-bnb-4bit"  # 2B model, fits T4 GPU
# LORA_BASE_MODEL = "unsloth/Qwen3-4B-unsloth-bnb-4bit"  # 4B model (needs A100)

# Dataset
DATASET_NAME = "yahma/alpaca-cleaned"  # Change to your dataset

# Training Mode
# Quick test (recommended for first run)
MAX_STEPS = 50              # Train for 50 steps only (~2 minutes)
DATASET_MAX_SAMPLES = 100   # Use 100 samples only

# Full training (uncomment to use)
# MAX_STEPS = 0               # Train for full epochs
# DATASET_MAX_SAMPLES = 0     # Use all samples

# Training Parameters
MAX_SEQ_LENGTH = 2048
LORA_RANK = 16              # Use 64 for production
LORA_ALPHA = 32             # Use 128 for production
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 1
WARMUP_STEPS = 2

# Output Formats (GGUF requires llama.cpp - not available in Colab)
OUTPUT_FORMATS = ""  # Empty = no GGUF conversion (recommended for Colab)
# OUTPUT_FORMATS = "gguf_q4_k_m"  # Only works with local setup

# Output naming
OUTPUT_MODEL_NAME = "auto"  # Auto-generate name

# Author
AUTHOR_NAME = "Your Name"  # Your name for model card

print("✅ Configuration set!")
print(f"Model: {LORA_BASE_MODEL}")
print(f"Dataset: {DATASET_NAME}")
print(f"Training: {MAX_STEPS} steps, {DATASET_MAX_SAMPLES} samples")

## Step 4: Create .env File

In [ ]:
# Create .env file with configuration
env_content = f"""
# Model
LORA_BASE_MODEL={LORA_BASE_MODEL}
INFERENCE_BASE_MODEL=
OUTPUT_MODEL_NAME={OUTPUT_MODEL_NAME}

# Dataset
DATASET_NAME={DATASET_NAME}
DATASET_MAX_SAMPLES={DATASET_MAX_SAMPLES}
MAX_STEPS={MAX_STEPS}

# Training
MAX_SEQ_LENGTH={MAX_SEQ_LENGTH}
LORA_RANK={LORA_RANK}
LORA_ALPHA={LORA_ALPHA}
BATCH_SIZE={BATCH_SIZE}
GRADIENT_ACCUMULATION_STEPS={GRADIENT_ACCUMULATION_STEPS}
LEARNING_RATE={LEARNING_RATE}
NUM_TRAIN_EPOCHS={NUM_TRAIN_EPOCHS}
WARMUP_STEPS={WARMUP_STEPS}
PACKING=false

# Optimization
USE_GRADIENT_CHECKPOINTING=true
MAX_GRAD_NORM=1.0
OPTIM=adamw_8bit

# Logging
LOGGING_STEPS=5
SAVE_STEPS=25
SAVE_TOTAL_LIMIT=2
SAVE_ONLY_FINAL=true

# Monitoring
WANDB_ENABLED=false

# Output
OUTPUT_FORMATS={OUTPUT_FORMATS}
OUTPUT_DIR_BASE=./outputs
PREPROCESSED_DATA_DIR=./data/preprocessed
CACHE_DIR=./cache

# HuggingFace
PUSH_TO_HUB=false
HF_USERNAME=your_username
HF_MODEL_NAME=auto
HF_TOKEN=

# Author
AUTHOR_NAME={AUTHOR_NAME}

# Advanced
SEED=3407
FORCE_PREPROCESS=false
FORCE_RETRAIN=true
FORCE_REBUILD=true
CHECK_SEQ_LENGTH=false
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("✅ .env file created!")

## Step 5: Train Model

This will take ~2 minutes for quick test, or hours for full training.

In [ ]:
# Run training
!python train.py

## Step 6: Build Merged Model

This creates the merged model (LoRA + base model combined) in safetensors format.

**Why skip GGUF in Colab?**
- GGUF conversion requires llama.cpp (not available in Colab)
- **Better workflow:** Create merged model here, then convert to GGUF locally (CPU-only, no GPU needed)

**This step creates:** `merged_16bit/` folder with complete model in safetensors format

In [ ]:
# Build merged model (safetensors format)
# This skips GGUF since OUTPUT_FORMATS is empty
!python build.py

## Step 7: Save Your Model

**You have two models to save:**

1. **LoRA adapters** (~80-100MB) - Small, efficient, requires base model to use
2. **Merged model** (~5GB for 2B) - Complete model, ready to use anywhere

**Choose your preferred method:**
- **Option A (Recommended):** HuggingFace Hub - Free, unlimited storage, easy sharing
- **Option B:** Google Drive - Simple, but limited free storage (15GB)

In [ ]:
# Check your model output
import os

# List output directories
output_dirs = [d for d in os.listdir('outputs') if os.path.isdir(os.path.join('outputs', d))]
if output_dirs:
    model_dir = output_dirs[0]
    print(f"✅ Your model is in: outputs/{model_dir}")
    print(f"\nContents:")
    !ls -lh outputs/{model_dir}
    print(f"\nLoRA adapters: outputs/{model_dir}/lora/")
    print(f"Merged model: outputs/{model_dir}/merged_16bit/")
else:
    print("❌ No model found in outputs/")

# Option 2: Download via Colab Files (for small files only)
from google.colab import files

# Download LoRA adapters (small, ~80MB)
print("Zipping LoRA adapters...")
!zip -r lora_adapters.zip outputs/*/lora/
print("Downloading... (this may take a minute)")
files.download('lora_adapters.zip')
print("✅ LoRA adapters downloaded!")
print("")
print("⚠️  Note: Merged model (~5GB) is too large for direct download")
print("   Use Option 1 (HuggingFace) or Option 3 (Google Drive) instead")

In [ ]:
# Option 3: Upload to Google Drive (if you have space)
from google.colab import drive
drive.mount('/content/drive')

# Copy to Google Drive
!mkdir -p /content/drive/MyDrive/unsloth-models
!cp -r outputs/* /content/drive/MyDrive/unsloth-models/

print("✅ Model copied to Google Drive: MyDrive/unsloth-models/")
print("")
print("📁 Your model contains:")
print("   - lora/ - LoRA adapters (~80MB)")
print("   - merged_16bit/ - Merged model in safetensors format (~5GB for 2B model)")
print("")
print("⚠️  Note: This uses ~5GB of your Google Drive storage")
print("   Consider using HuggingFace Hub (Option 1) for unlimited free storage")

In [ ]:
# Option 1: Push to HuggingFace Hub (RECOMMENDED - Free unlimited storage)
!pip install -q huggingface_hub

from huggingface_hub import login, HfApi, create_repo
import os

# Login to HuggingFace (you'll need a token from https://huggingface.co/settings/tokens)
print("Get your token from: https://huggingface.co/settings/tokens")
print("Select 'Write' permission when creating the token")
print("")
login()

# Find your model directory
output_dirs = [d for d in os.listdir('outputs') if os.path.isdir(os.path.join('outputs', d))]
if not output_dirs:
    print("❌ No model found in outputs/")
else:
    model_dir = output_dirs[0]
    
    # Your HuggingFace username (will be detected after login)
    api = HfApi()
    user_info = api.whoami()
    username = user_info['name']
    
    print(f"✅ Logged in as: {username}")
    print("")
    
    # Upload LoRA adapters
    lora_repo_name = f"{model_dir}-lora"
    lora_repo_id = f"{username}/{lora_repo_name}"
    
    print(f"📤 Uploading LoRA adapters to: {lora_repo_id}")
    try:
        create_repo(repo_id=lora_repo_id, repo_type="model", exist_ok=True)
        api.upload_folder(
            folder_path=f"outputs/{model_dir}/lora",
            repo_id=lora_repo_id,
            repo_type="model"
        )
        print(f"✅ LoRA adapters uploaded: https://huggingface.co/{lora_repo_id}")
    except Exception as e:
        print(f"❌ LoRA upload failed: {e}")
    
    print("")
    
    # Upload merged model
    merged_repo_name = f"{model_dir}"
    merged_repo_id = f"{username}/{merged_repo_name}"
    
    print(f"📤 Uploading merged model to: {merged_repo_id}")
    print("⏳ This may take 5-10 minutes for a 5GB model...")
    try:
        create_repo(repo_id=merged_repo_id, repo_type="model", exist_ok=True)
        api.upload_folder(
            folder_path=f"outputs/{model_dir}/merged_16bit",
            repo_id=merged_repo_id,
            repo_type="model"
        )
        print(f"✅ Merged model uploaded: https://huggingface.co/{merged_repo_id}")
    except Exception as e:
        print(f"❌ Merged model upload failed: {e}")
    
    print("")
    print("=" * 60)
    print("✅ UPLOAD COMPLETE!")
    print("=" * 60)
    print("")
    print("📦 Your models:")
    print(f"   LoRA:   https://huggingface.co/{lora_repo_id}")
    print(f"   Merged: https://huggingface.co/{merged_repo_id}")
    print("")
    print("Next: Download merged model to convert to GGUF locally")

### Option B: Google Drive (Alternative)

In [ ]:
# B1. Upload to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy to Google Drive
!mkdir -p /content/drive/MyDrive/unsloth-models
!cp -r outputs/* /content/drive/MyDrive/unsloth-models/

print("✅ Model copied to Google Drive: MyDrive/unsloth-models/")
print("")
print("📁 Your model contains:")
print("   - lora/ - LoRA adapters (~80MB)")
print("   - merged_16bit/ - Merged model in safetensors format (~5GB for 2B model)")
print("")
print("⚠️  Note: Google Drive free tier has 15GB storage limit")
print("Next: Download from Google Drive to convert to GGUF locally")

## Step 8: Convert to GGUF Locally (Optional)

After saving your model (HuggingFace or Google Drive), convert to GGUF on your local machine.

**This works even with limited VRAM** - GGUF conversion is CPU-only, no GPU needed!

### Option A: Download from HuggingFace and Convert

```bash
# 1. On your local machine, navigate to unsloth-finetuning
cd /path/to/unsloth-finetuning

# 2. Download merged model from HuggingFace
huggingface-cli download your-username/model-name-merged \
  --local-dir outputs/model-name-merged

# 3. Update .env to enable GGUF conversion
# Edit .env and set:
OUTPUT_FORMATS=gguf_q4_k_m

# 4. Run build.py (will only do GGUF conversion)
python build.py
```

### Option B: Download from Google Drive and Convert

```bash
# 1. Download model from Google Drive to your local machine
# Location: MyDrive/unsloth-models/Qwen3-VL-2B-Instruct-alpaca-cleaned/

# 2. Navigate to your local unsloth-finetuning directory
cd /path/to/unsloth-finetuning

# 3. Copy merged model to outputs folder
cp -r ~/Downloads/Qwen3-VL-2B-Instruct-alpaca-cleaned outputs/

# 4. Update .env to enable GGUF conversion
# Edit .env and set:
OUTPUT_FORMATS=gguf_q4_k_m

# 5. Run build.py (will only do GGUF conversion since merged model exists)
python build.py
```

**Result:** You'll get `outputs/model-name/gguf/model.Q4_K_M.gguf`

**Alternative - Direct llama.cpp conversion:**
```bash
# If you already ran setup.sh locally
./llama.cpp/llama-quantize \
  outputs/model-name/merged_16bit/model.safetensors \
  outputs/model-name/gguf/model.Q4_K_M.gguf \
  q4_k_m
```

## Step 9: Test Your Model (Optional)

Quick test of your fine-tuned model:

In [ ]:
from unsloth import FastLanguageModel
import torch

# Load your fine-tuned model
model_path = f"outputs/{model_dir}/lora"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)

# Test prompt
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is machine learning?

### Response:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n" + "="*50)
print("MODEL RESPONSE:")
print("="*50)
print(response)
print("="*50)

## 🎉 Done!

Your model has been trained and is ready to use!

**Next steps:**
1. Download the model from Google Drive or HuggingFace
2. Use it locally with Ollama or transformers
3. Share it on HuggingFace Hub

**Resources:**
- [Documentation](https://github.com/farhan-syah/unsloth-finetuning/tree/main/docs)
- [Training Guide](https://github.com/farhan-syah/unsloth-finetuning/blob/main/docs/TRAINING.md)
- [FAQ](https://github.com/farhan-syah/unsloth-finetuning/blob/main/docs/FAQ.md)